# Question Generator (QG) Model Using HuggingFace T5

## Install and Import Needed Library

In [ ]:
!pip install transformers datasets sentencepiece

In [ ]:
!pip install transformers seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=806845b3c46655f96a8b89d10fb6255a6033e511b644b87017646ee65df3d28e
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [2]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=e1346d2080fc6469f5da7a8819631ab024523637d89506228c49e204a5805495
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [3]:
from google.colab import files
import pandas as pd
import random
from datasets import Dataset
import torch
import re
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from rouge_score import rouge_scorer

## Data Loading

In [4]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"meilanikizana","key":"230db92153864ff31f16103462e6c659"}'}

In [5]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d mintupsidup/squad-variated-indo
!unzip squad-variated-indo.zip

Dataset URL: https://www.kaggle.com/datasets/mintupsidup/squad-variated-indo
License(s): apache-2.0
 86% 88.0M/103M [00:00<00:00, 919MB/s]
100% 103M/103M [00:00<00:00, 922MB/s] 
Archive:  squad-variated-indo.zip
  inflating: dev-v2.0.json           
  inflating: glove_50d_Indo.txt      
  inflating: train-v2.0.json         
  inflating: train_df_variated_new.csv  
  inflating: valid_df_variated_new.csv  


In [7]:
train = pd.read_json('train-v2.0.json', orient='column')
dev = pd.read_json('dev-v2.0.json', orient='column')

init_df = pd.concat([train, dev], ignore_index=True)
init_df

,version,data
0,v2.0,"{'title': 'Beyoncé', 'paragraphs': [{'context'..."
1,v2.0,"{'title': 'Frédéric_Chopin', 'paragraphs': [{'..."
2,v2.0,{'title': 'Sino-Tibetan_relations_during_the_M...
3,v2.0,"{'title': 'IPod', 'paragraphs': [{'context': '..."
4,v2.0,{'title': 'The_Legend_of_Zelda:_Twilight_Princ...
...,...,...
472,v2.0,"{'title': 'Islamism', 'paragraphs': [{'context..."
473,v2.0,"{'title': 'Imperialism', 'paragraphs': [{'cont..."
474,v2.0,"{'title': 'Warsaw', 'paragraphs': [{'context':..."
475,v2.0,"{'title': 'French_and_Indian_War', 'paragraphs..."


In [8]:
init_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 477 entries, 0 to 476
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   version  477 non-null    object
 1   data     477 non-null    object
dtypes: object(2)
memory usage: 7.6+ KB


## Data Preparation

### Extract Features

In [9]:
contexts, questions, answers = [], [], []

for data in init_df['data']:
    for para in data['paragraphs']:
        context = para['context']
        for all_questions in para['qas']:
            if not all_questions.get("is_impossible", False):
                question = all_questions['question']
                answer_list = all_questions['answers']
                if len(answer_list) > 0:
                    answer = answer_list[0]['text']
                    questions.append(question)
                    contexts.append(context)
                    answers.append(answer)

print(f"Total data pertanyaan: {len(questions)}")

Total data pertanyaan: 92749


In [10]:
df = pd.DataFrame({
    "context": contexts,
    "question": questions,
    "answer": answers
})

df

,context,question,answer
0,Beyoncé Giselle Knowles-Carter (/bijanse / bee...,Kapan Beyonce mulai menjadi populer?,pada akhir 1990-an
1,Beyoncé Giselle Knowles-Carter (/bijanse / bee...,Daerah mana saja yang bersaing dengan Beyonce ...,menyanyi dan
2,Beyoncé Giselle Knowles-Carter (/bijanse / bee...,Kapan Beyonce meninggalkan Destiny's Child dan...,2003
3,Beyoncé Giselle Knowles-Carter (/bijanse / bee...,Di kota dan negara mana Beyonce tumbuh?,"Houston, Texas"
4,Beyoncé Giselle Knowles-Carter (/bijanse / bee...,Pada dekade mana Beyonce menjadi terkenal?,akhir 1990-an
...,...,...,...
92744,"Pound-force memiliki mitra metrik, kurang umum...",Apa istilah metrik yang kurang digunakan dari ...,kilogram-force
92745,"Pound-force memiliki mitra metrik, kurang umum...",Apa daya kilogram kadang-kadang direffer ke se...,kilopond
92746,"Pound-force memiliki mitra metrik, kurang umum...",Apa yang jarang digunakan unit massa dalam sis...,siput
92747,"Pound-force memiliki mitra metrik, kurang umum...",Apa yang jarang digunakan istilah unit kekuata...,kip


### Handling Missing Values and Duplicated Data

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92749 entries, 0 to 92748
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   context   92749 non-null  object
 1   question  92749 non-null  object
 2   answer    92749 non-null  object
dtypes: object(3)
memory usage: 2.1+ MB


In [12]:
print("Missing values:\n", df.isnull().sum())
print("\nDuplikasi baris:", df.duplicated().sum())

Missing values:
 context     0
question    0
answer      0
dtype: int64

Duplikasi baris: 55


In [13]:
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)
df

,context,question,answer
0,Beyoncé Giselle Knowles-Carter (/bijanse / bee...,Kapan Beyonce mulai menjadi populer?,pada akhir 1990-an
1,Beyoncé Giselle Knowles-Carter (/bijanse / bee...,Daerah mana saja yang bersaing dengan Beyonce ...,menyanyi dan
2,Beyoncé Giselle Knowles-Carter (/bijanse / bee...,Kapan Beyonce meninggalkan Destiny's Child dan...,2003
3,Beyoncé Giselle Knowles-Carter (/bijanse / bee...,Di kota dan negara mana Beyonce tumbuh?,"Houston, Texas"
4,Beyoncé Giselle Knowles-Carter (/bijanse / bee...,Pada dekade mana Beyonce menjadi terkenal?,akhir 1990-an
...,...,...,...
92689,"Pound-force memiliki mitra metrik, kurang umum...",Apa istilah metrik yang kurang digunakan dari ...,kilogram-force
92690,"Pound-force memiliki mitra metrik, kurang umum...",Apa daya kilogram kadang-kadang direffer ke se...,kilopond
92691,"Pound-force memiliki mitra metrik, kurang umum...",Apa yang jarang digunakan unit massa dalam sis...,siput
92692,"Pound-force memiliki mitra metrik, kurang umum...",Apa yang jarang digunakan istilah unit kekuata...,kip


In [14]:
print("Duplikasi baris:", df.duplicated().sum())

Duplikasi baris: 0


### Data Sampling, Convert to HuggingFace Dataset & Split to Train and Test

In [15]:
sample_size = 1000
df_sample = df.sample(n=sample_size, random_state=42).reset_index(drop=True)

contexts_list = df_sample['context'].tolist()
questions_list = df_sample['question'].tolist()
answers_list = df_sample['answer'].tolist()

raw_dataset = Dataset.from_dict({
    "context": contexts_list,
    "question": questions_list,
    "answer": answers_list
})

raw_dataset = raw_dataset.train_test_split(test_size=0.2)
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'answer'],
        num_rows: 800
    })
    test: Dataset({
        features: ['context', 'question', 'answer'],
        num_rows: 200
    })
})

### Preprocessing and Tokenization

In [16]:
tokenizer = T5Tokenizer.from_pretrained("cahya/t5-base-indonesian-summarization-cased")

def preprocess(text):
    input_text = f"generate question: context: {text['context']} answer: {text['answer']}"
    target_text = text["question"]

    model_inputs = tokenizer(
        input_text, max_length=512, padding="max_length", truncation=True
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            target_text, max_length=64, padding="max_length", truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


tokenized_dataset = raw_dataset.map(preprocess, batched=False)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/793k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/657 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/800 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

## Load and Configure T5 Model

In [17]:
t5_model = T5ForConditionalGeneration.from_pretrained("cahya/t5-base-indonesian-summarization-cased")

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [18]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=500,
    report_to="none",
    fp16=True
)

trainer = Trainer(
    model=t5_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
)

trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss


TrainOutput(global_step=200, training_loss=0.822532958984375, metrics={'train_runtime': 96.2696, 'train_samples_per_second': 8.31, 'train_steps_per_second': 2.077, 'total_flos': 487166312448000.0, 'train_loss': 0.822532958984375, 'epoch': 1.0})

## Load Indonesian Named Entity Recognition (NER) Model

In [19]:
ner_tokenizer = AutoTokenizer.from_pretrained("cahya/bert-base-indonesian-NER")
ner_model = AutoModelForTokenClassification.from_pretrained("cahya/bert-base-indonesian-NER")

ner_pipe = pipeline("ner", model=ner_model, tokenizer=ner_tokenizer, grouped_entities=True)

tokenizer_config.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/230k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of the model checkpoint at cahya/bert-base-indonesian-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/token_classification.py:170: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


## Functions

In [20]:
def extract_entities(text, max_entities=5):
    entities = ner_pipe(text)
    answers = []
    for entity in entities:
        if entity['entity_group'] in ['PER', 'LOC', 'DAT']:
            word = entity['word'].replace('##', '').strip()
            if word not in answers:
                answers.append(word)
        if len(answers) >= max_entities:
            break
    return answers

In [47]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
t5_model.to(device)

def get_sentence_with_answer(text, answer):
    sentences = re.split(r'(?<=[.!?]) +', text)
    for sentence in sentences:
        if answer in sentence:
            return sentence
    return text

def generate_question_from_input(input_text):
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    ).to(device)

    outputs = t5_model.generate(
        inputs["input_ids"],
        max_length=64,
        num_return_sequences=1,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.8
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def generate_questions(text, num_questions=5):
    answers = extract_entities(text, max_entities=num_questions)
    questions = []

    if answers:
        for ans in answers:
            context = get_sentence_with_answer(text, ans)
            input_text = f"generate question: context: {context} answer: {ans}"
            question = generate_question_from_input(input_text)
            questions.append(question)
            remaining_questions = num_questions - len(questions)
        if remaining_questions > 0:
            for i in range(remaining_questions):
                input_text = f"generate question: context: {text}"
                question = generate_question_from_input(input_text)
                questions.append(question)
    else:
        for _ in range(num_questions):
            input_text = f"generate question: context: {text}"
            question = generate_question_from_input(input_text)
            questions.append(question)

    return questions

## Inference Process

In [ ]:
text = "Hari lahir Pancasila jatuh pada tanggal 1 Juni yang ditandai oleh pidato yang dilakukan oleh Presiden pertama Indonesia Ir Soekarno pada tanggal 1 Juni 1945 dalam sidang Dokuritsu Junbi Cosakai (Badan Penyelidik Usaha Persiapan Kemerdekaan). Dalam pidatonya pertama kali mengemukakan konsep awal Pancasila yang menjadi dasar negara Indonesia. Adapun sejarahnya berawal dari kekalahan Jepang pada perang pasifik, mereka kemudian berusaha mendapatkan hati masyarakat dengan menjanjikan kemerdekaan kepada Indonesia dan membentuk sebuah Lembaga yang tugasnya untuk mempersiapkan hal tersebut. Lembaga ini dinamai Dokuritsu Junbi Cosakai. Pada sidang pertamanya di tanggal 29 Mei 1945 yang diadakan di Gedung Chuo Sangi In (sekarang Gedung Pancasila), para anggota membahas mengenai tema dasar negara. Sidang berjalan sekitar hampir 5 hari, kemudian pada tanggal 1 Juni 1945, Soekarno menyampaikan ide serta gagasannya terkait dasar negara Indonesia, yang dinamai “Pancasila”. Panca artinya lima, sedangkan sila artinya prinsip atau asas."
questions = generate_questions(text, num_questions=5)

for i, q in enumerate(questions, 1):
    print(f"{i}. {q}")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


1. Kapan Dokuritsu Junbi Cosakai memulai sidang pertamanya?
2. Apa yang dimaksudkan pada hari lahir Pancasila?
3. Hari ulang tahun untuk apa ini terjadi?
4. Apa yang disampaikan Presiden pertama Indonesia pada sidang Dokuritsu Junbi Cosakai?
5. Pada tanggal 1 Juni 1945, Presiden pertama Indonesia Ir Soekarno duduk dalam sidang Dokuritsu Junbi Cosakai.


## Evaluation

In [22]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

### Data Train 500

In [23]:
sample_data = raw_dataset['train'][500]
context_500 = sample_data['context']
reference_question_500 = sample_data['question']

print("Konteks:\n", context_500)
print("\nPertanyaan:\n", reference_question_500)

Konteks:
 Populasi Inggris meningkat pesat selama abad ke - 19 dan ke - 20 sedangkan populasi Skotlandia dan Wales tidak banyak meningkat selama abad ke - 20, dengan populasi Skotlandia yang tersisa tidak berubah sejak 1951. Irlandia untuk sebagian besar sejarahnya terdiri atas penduduk yang proporsional dengan daerah daratannya (sekitar sepertiga dari total populasi). Akan tetapi, karena Bala Kelaparan Irlandia Besar, populasi Irlandia telah merosot hingga kurang dari sepersepuluh penduduk Kepulauan Inggris. Kelaparan, yang menyebabkan penurunan populasi selama berabad-abad, secara drastis mengurangi populasi Irlandia dan secara permanen mengubah susunan demografis dari Kepulauan Inggris. Dalam skala global, bencana ini menyebabkan penciptaan diaspora Irlandia yang berjumlah 15 kali populasi pulau ini saat ini.

Pertanyaan:
 Yang dua daerah di Kepulauan Inggris belum mengalami pertumbuhan populasi besar dalam seratus tahun terakhir?


In [48]:
questions_500 = generate_questions(context_500, num_questions=5)
generated_question_500 = []

for i, q in enumerate(questions_500, 1):
    generated_question_500.append(q)
    print(f"{i}. {q}")

1. Populasi Inggris dari seluruh dunia?
2. Apa yang menyebabkan pengurangan populasi?
3. Apa yang menyebabkan populasi pulau Irlandia menurun dan menyusut selama berabad-abad?
4. Apa yang terjadi pada Kepulauan Inggris?
5. Apa masalah Irlandia dan Kepulauan Inggris?


In [49]:
for i, q in enumerate(generated_question_500, 1):
    scores = scorer.score(reference_question_500, q)
    print(f"{i}. {scores}")

1. {'rouge1': Score(precision=0.4, recall=0.13333333333333333, fmeasure=0.2), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.2, recall=0.06666666666666667, fmeasure=0.1)}
2. {'rouge1': Score(precision=0.4, recall=0.13333333333333333, fmeasure=0.2), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.4, recall=0.13333333333333333, fmeasure=0.2)}
3. {'rouge1': Score(precision=0.16666666666666666, recall=0.13333333333333333, fmeasure=0.14814814814814814), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.16666666666666666, recall=0.13333333333333333, fmeasure=0.14814814814814814)}
4. {'rouge1': Score(precision=0.5, recall=0.2, fmeasure=0.28571428571428575), 'rouge2': Score(precision=0.2, recall=0.07142857142857142, fmeasure=0.10526315789473682), 'rougeL': Score(precision=0.5, recall=0.2, fmeasure=0.28571428571428575)}
5. {'rouge1': Score(precision=0.3333333333333333, recall=0.13333

### Data Train 250

In [54]:
sample_data = raw_dataset['train'][250]
context_250 = sample_data['context']
reference_question_250 = sample_data['question']

print("Konteks:\n", context_250)
print("\nPertanyaan:\n", reference_question_250)

Konteks:
 Compact Disc adalah evolusi teknologi Laser Disc, di mana sinar laser yang terfokus digunakan yang memungkinkan kepadatan informasi tinggi yang diperlukan untuk sinyal audio digital berkualitas tinggi. Prototipe dikembangkan oleh Philips dan Sony secara independen pada akhir 1970-an. Pada tahun 1979, Sony dan Philips mendirikan satuan tugas gabungan para insinyur untuk merancang cakram audio digital yang baru. Setelah satu tahun eksperimen dan diskusi, standar CD-DA Buku Merah diterbitkan pada tahun 1980. Setelah rilis komersial mereka pada tahun 1982, compact disc dan pemain mereka sangat populer. Meskipun harganya mencapai 1.000 dolar, lebih dari 400.000 pemutar CD dijual di Amerika Serikat antara tahun 1983 dan 1984. Keberhasilan cakram compact telah dianggap sebagai hasil kerja sama antara Philips dan Sony, yang datang bersama - sama untuk menyetujui dan mengembangkan perangkat keras yang kompatibel. Desain terpadu dari cakram compact memungkinkan konsumen membeli cakram 

In [55]:
questions_250 = generate_questions(context_250, num_questions=5)
generated_question_250 = []

for i, q in enumerate(questions_250, 1):
    generated_question_250.append(q)
    print(f"{i}. {q}")

1. Kapan The Stungs?
2. Pada tahun 1979, Sony dan Philips mendirikan sebuah negara yang menggabungkan insinyur untuk menyusun cakram audio digital baru.
3. Berapa lama standar CD-DA Buku Merah diterbitkan?
4. Apa yang dishec bermain disc pada tahun 1982?
5. Berapa banyak pemutar CD yang dijual di Amerika Serikat antara tahun 1983 dan 1984?


In [56]:
for i, q in enumerate(generated_question_250, 1):
    scores = scorer.score(reference_question_250, q)
    print(f"{i}. {scores}")

1. {'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.0, recall=0.0, fmeasure=0.0)}
2. {'rouge1': Score(precision=0.1111111111111111, recall=0.18181818181818182, fmeasure=0.13793103448275862), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.05555555555555555, recall=0.09090909090909091, fmeasure=0.06896551724137931)}
3. {'rouge1': Score(precision=0.125, recall=0.09090909090909091, fmeasure=0.10526315789473685), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.125, recall=0.09090909090909091, fmeasure=0.10526315789473685)}
4. {'rouge1': Score(precision=0.375, recall=0.2727272727272727, fmeasure=0.3157894736842105), 'rouge2': Score(precision=0.14285714285714285, recall=0.1, fmeasure=0.11764705882352941), 'rougeL': Score(precision=0.375, recall=0.2727272727272727, fmeasure=0.3157894736842105)}
5. {'rouge1': Score(precision

### Data Train 150

In [63]:
sample_data = raw_dataset['train'][150]
context_150 = sample_data['context']
reference_question_150 = sample_data['question']

print("Konteks:\n", context_150)
print("\nPertanyaan:\n", reference_question_150)

Konteks:
 BBC juga memperkenalkan Ceefax, layanan teleteks pertama, mulai tahun 1974. Layanan ini memungkinkan pemirsa BBC untuk melihat informasi tekstual seperti berita terbaru di televisi mereka. CEEFAX belum membuat transisi penuh ke televisi digital, alih-alih digantikan oleh layanan BBCi interaktif baru.

Pertanyaan:
 Kapan Ceefax diluncurkan?


In [64]:
questions_150 = generate_questions(context_150, num_questions=5)
generated_question_150 = []

for i, q in enumerate(questions_150, 1):
    generated_question_150.append(q)
    print(f"{i}. {q}")

1. Siapa penyedia telekomunikasi pertama?
2. Apa nama Ceefax?
3. Di mana CEEFMX diperkenalkan?
4. Kapan Ceefax diperkenalkan?
5. Siapa yang "terdebat sepenuhnya ke televisi digital?"


In [65]:
for i, q in enumerate(generated_question_150, 1):
    scores = scorer.score(reference_question_150, q)
    print(f"{i}. {scores}")

1. {'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.0, recall=0.0, fmeasure=0.0)}
2. {'rouge1': Score(precision=0.3333333333333333, recall=0.3333333333333333, fmeasure=0.3333333333333333), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.3333333333333333, recall=0.3333333333333333, fmeasure=0.3333333333333333)}
3. {'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.0, recall=0.0, fmeasure=0.0)}
4. {'rouge1': Score(precision=0.6666666666666666, recall=0.6666666666666666, fmeasure=0.6666666666666666), 'rouge2': Score(precision=0.5, recall=0.5, fmeasure=0.5), 'rougeL': Score(precision=0.6666666666666666, recall=0.6666666666666666, fmeasure=0.6666666666666666)}
5. {'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rouge2': Score(precision=0.0, recall=0.0, fme

### Data Train 50

In [68]:
sample_data = raw_dataset['train'][50]
context_50 = sample_data['context']
reference_question_50 = sample_data['question']

print("Konteks:\n", context_50)
print("\nPertanyaan:\n", reference_question_50)

Konteks:
 Gletser membentuk di mana akumulasi salju dan es melebihi ablasi. Daerah di mana bentuk gletser disebut sirque (korie atau cwm) - yang biasanya berbentuk kursi fitur geologi (seperti depresi antara gunung yang tertutup oleh arêtes) - yang mengumpulkan dan memampatkan melalui gravitasi salju yang jatuh ke dalamnya. Salju ini mengumpulkan dan dipadatkan oleh berat salju yang jatuh di atasnya membentuk névé. Lebih menghancurkan setiap kepingan salju dan meremas udara dari salju mengubahnya menjadi 'es glocial'. Es glacial ini akan mengisi Cirque sampai 'overflows' melalui kelemahan geologi atau kekosongan, seperti kesenjangan antara dua gunung. Ketika massa salju dan es cukup tebal, mulai bergerak karena kombinasi lereng permukaan, gravitasi dan tekanan. Di lereng yang curam, ini dapat terjadi dengan hanya 15 meter (50 kaki) es salju.

Pertanyaan:
 Apa itu Cirque?


In [71]:
questions_50 = generate_questions(context_50, num_questions=5)
generated_question_50 = []

for i, q in enumerate(questions_50, 1):
    generated_question_50.append(q)
    print(f"{i}. {q}")

1. Apa yang paling kerap digunakan sebagai tempat penimbangan salju?
2. Apa yang di mana sirque atau cwm disebut?
3. Di mana gletser disebut sirque, atau Cwm?
4. Apa arti es glocial?
5. Apa itu es glacial?


In [72]:
for i, q in enumerate(generated_question_50, 1):
    scores = scorer.score(reference_question_50, q)
    print(f"{i}. {scores}")

1. {'rouge1': Score(precision=0.1111111111111111, recall=0.3333333333333333, fmeasure=0.16666666666666666), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.1111111111111111, recall=0.3333333333333333, fmeasure=0.16666666666666666)}
2. {'rouge1': Score(precision=0.125, recall=0.3333333333333333, fmeasure=0.18181818181818182), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.125, recall=0.3333333333333333, fmeasure=0.18181818181818182)}
3. {'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.0, recall=0.0, fmeasure=0.0)}
4. {'rouge1': Score(precision=0.25, recall=0.3333333333333333, fmeasure=0.28571428571428575), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.25, recall=0.3333333333333333, fmeasure=0.28571428571428575)}
5. {'rouge1': Score(precision=0.5, recall=0.6666666666666666, fmea

### Data Train 5

In [78]:
sample_data = raw_dataset['train'][5]
context_5 = sample_data['context']
reference_question_5 = sample_data['question']

print("Konteks:\n", context_5)
print("\nPertanyaan:\n", reference_question_5)

Konteks:
 Teknik pemisahan modern seperti menggiling, centrifugasi, dan menekan telah memungkinkan konsentrasi komponen makanan tertentu, menghasilkan tepung, minyak, jus, dan sebagainya, dan bahkan asam lemak yang terpisah, asam amino, vitamin, dan mineral. Tak pelak lagi, konsentrasi besar-besaran ini mengubah kandungan gizi makanan, menyimpan nutrisi tertentu sambil menyingkirkan yang lain. Teknik Heating juga dapat mengurangi kandungan makanan dari banyak nutrisi panas-labile seperti vitamin tertentu dan fitokimia, dan mungkin lainnya belum ditemukan zat. Karena nilai gizi yang berkurang, makanan olahan sering kali 'dikayakan' atau 'dibentengi' dengan beberapa nutrisi yang paling penting (biasanya vitamin tertentu) yang hilang selama proses. Meskipun demikian, makanan olahan cenderung memiliki profil gizi rendah dibandingkan dengan makanan yang utuh dan segar, sehubungan dengan kandungan baik gula maupun tinggi GI pati, kalium/sodium, vitamin, serat, dan asam lemak yang utuh, tidak

In [79]:
questions_5 = generate_questions(context_5, num_questions=5)
generated_question_5 = []

for i, q in enumerate(questions_5, 1):
    generated_question_5.append(q)
    print(f"{i}. {q}")

1. Makan apa yang mendapat manfaat?
2. Bagaimana teknik pemisahan bisnis?
3. Apa yang disebut "penggunaan yang paling berkesan"?
4. Apa yang membuat makanan olahan mengandung zat-zat apa?
5. Apa yang dilakukan dengan cara ini?


In [80]:
for i, q in enumerate(generated_question_5, 1):
    scores = scorer.score(reference_question_5, q)
    print(f"{i}. {scores}")

1. {'rouge1': Score(precision=0.4, recall=0.14285714285714285, fmeasure=0.21052631578947364), 'rouge2': Score(precision=0.25, recall=0.07692307692307693, fmeasure=0.11764705882352941), 'rougeL': Score(precision=0.4, recall=0.14285714285714285, fmeasure=0.21052631578947364)}
2. {'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.0, recall=0.0, fmeasure=0.0)}
3. {'rouge1': Score(precision=0.42857142857142855, recall=0.21428571428571427, fmeasure=0.2857142857142857), 'rouge2': Score(precision=0.16666666666666666, recall=0.07692307692307693, fmeasure=0.10526315789473684), 'rougeL': Score(precision=0.2857142857142857, recall=0.14285714285714285, fmeasure=0.19047619047619047)}
4. {'rouge1': Score(precision=0.3333333333333333, recall=0.21428571428571427, fmeasure=0.2608695652173913), 'rouge2': Score(precision=0.125, recall=0.07692307692307693, fmeasure=0.09523809523809525), 'rougeL': Score(precision=0

## Save Pretrained Model and Tokenizer

In [81]:
t5_model.save_pretrained("qg_model")
tokenizer.save_pretrained("qg_model")

('qg_model/tokenizer_config.json',
 'qg_model/special_tokens_map.json',
 'qg_model/spiece.model',
 'qg_model/added_tokens.json')